# Using lists to expand equations to many dimensions 

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

from modelclass import model
from modelmanipulation import Mexplode

## LIST command, define list and sublists    

Lists are used to direct how the do command explodes a template model. 
And to define sum functions, and for matrix packing and unpacking. 

``` 
List <listname> = <sublistname0> : <sl00> <sl01> .. <sl0n> / 
                     <sublistname1> : <sl10> <sl11> .. <sl1n> /
                     .............................]
                     $
```
**There should always be the same number of elements in sublists**

Example with one sublist: 

```
LIST BANK = BANK : ATBAW DEHAS DEHSH LUPCK DELBY DEVWF ESSAB  $
```

Example with two sublist: 

```
LIST CRCOUNTRY  =  CRCOUNTRY   : UK DE CH NL BE CA JP KY LU HK  / 
                  CRMODELGEO  : UK DE CH NL BE CA JP R20 LU HK $
``` 

**If a list name is created several times, the first time is the used.** This makes it more easy to make prototypes for a few banks. One just stick a short list with the same name in front of the main specification 


To process model specifications a class `Mexplode` is used. It will process specifications of loops and  sums, then it will normalize and enhance  the frml's so they look like. If so asked it will also create additional frmls to calculate:

- values without 

Normalization is a text processing step handling terms like dlog(), diff() at the right hand side or the left hand side. And 
also make sure only one variable is at the left hand side

Note: 
´Mexplode´ don't handle matrix expressions or  specifications where 

> FRML <frml options> endogenous variable = right hand side $

A frml looks like this

> FRML <frml options> Left hand side = right hand side $

However to 

In [ ]:


lists = '''
list BANKS = BANKS    : Danske  Nordea Ibs /
               country : denmark sweden denmark  
               

list ports = ports  : households   NFC    RE  sov  /
               reg  :       AIRB  AIRB   STA  STA 

list country__danske = loancountry : uk  DK IR 
list country__nordea = loancountry:  SE  DK 
list country__ibs    = loancountry:    DK ir


'''

## DO ...  ENDDO, loop over lists   

```
do <listname> $
stuff 
enddo $
```

Will take take the stuff and make a copy for each element of the sublists in list name. 

In [5]:
mtest = ''' 
do banks 
     profit__{banks} = revenue__{banks} - expenses__{banks} 
enddo  
'''
Mexplode(lists + mtest).show


Before doable
<SUM=ABE,HEST> [BANKS=COUNTRY = DENMARK ] LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS}
do_indicies_dict={'BANKS': 'COUNTRY  =  DENMARK ', 'SECTORS': None}*******************


After doable
DO BANKS COUNTRY  =  DENMARK  $ DO SECTORS  $
  FRML    <SUM=ABE,HEST> LOSS__{BANKS}__{SECTORS} =HOLDING__{BANKS}__{SECTORS} * PD__{BANKS}__{SECTORS} $
ENDDO $ ENDDO $ 
FRML <SUM=ABE,HEST> LOSS__ABE = SUM(BANKS COUNTRY  =  DENMARK ,SUM(SECTORS,LOSS__{BANKS}__{SECTORS})) $ 


FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $




In [6]:
mtest = ''' 
doable profit__{banks} = revenue_{banks} - expenses_{banks} 
'''
Mexplode(lists + mtest).show           


FRML <> PROFIT__DANSKE = REVENUE_DANSKE-EXPENSES_DANSKE $
FRML <> PROFIT__NORDEA = REVENUE_NORDEA-EXPENSES_NORDEA $
FRML <> PROFIT__IBS = REVENUE_IBS-EXPENSES_IBS $




### Use of sublist to inject additional information 

In [7]:
mtest = ''' 
do banks 
    profit__{banks} = revenue__{banks} - expenses__{banks} 
    expenses__{banks} = factor__{country} * revenue__{banks} 
enddo  
'''
Mexplode(lists+mtest).show

FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> EXPENSES__DANSKE = FACTOR__DENMARK*REVENUE__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> EXPENSES__NORDEA = FACTOR__SWEDEN*REVENUE__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <> EXPENSES__IBS = FACTOR__DENMARK*REVENUE__IBS $




In [8]:
mtest = ''' 
doable profit__{banks} = revenue__{banks} - expenses__{banks} 
doable expenses__{banks} = factor__{country} * revenue__{banks} 
'''
Mexplode(lists+mtest).show



FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <> EXPENSES__DANSKE = FACTOR__DENMARK*REVENUE__DANSKE $
FRML <> EXPENSES__NORDEA = FACTOR__SWEDEN*REVENUE__NORDEA $
FRML <> EXPENSES__IBS = FACTOR__DENMARK*REVENUE__IBS $




### Use of sublist to create conditional expressions 

In [9]:
mtest = ''' 

do banks country = denmark
    frml <> profit__{banks} = revenue__{banks} - expenses__{banks} 
    frml <> expenses__{banks} = factor__{country} * revenue__{banks} 
enddo  
'''

xx = Mexplode(lists+mtest) 
xx.show  


FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> EXPENSES__DANSKE = FACTOR__DENMARK*REVENUE__DANSKE $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <> EXPENSES__IBS = FACTOR__DENMARK*REVENUE__IBS $




In [10]:
mtest = ''' 
    doable <> [country = denmark]  profit__{banks} = revenue__{banks} - expenses__{banks} 
    doable <> [country = denmark] expenses__{banks} = factor__{country} * revenue__{banks} 
'''

xx = Mexplode(lists+mtest) 
xx.show  




FRML <> PROFIT__DANSKE = REVENUE__DANSKE-EXPENSES__DANSKE $
FRML <> PROFIT__NORDEA = REVENUE__NORDEA-EXPENSES__NORDEA $
FRML <> PROFIT__IBS = REVENUE__IBS-EXPENSES__IBS $
FRML <> EXPENSES__DANSKE = FACTOR__DENMARK*REVENUE__DANSKE $
FRML <> EXPENSES__NORDEA = FACTOR__SWEDEN*REVENUE__NORDEA $
FRML <> EXPENSES__IBS = FACTOR__DENMARK*REVENUE__IBS $




### Nested do

Do loops can be nested.

One  have to take care that there is no duplication of sublist names. 

In [11]:
mtest = ''' 
    doable  loss__{ports}__{banks} = 0.02 * stock__{ports}__{banks} 
'''
Mexplode(lists + mtest).show


FRML <> LOSS__HOUSEHOLDS__DANSKE = 0.02*STOCK__HOUSEHOLDS__DANSKE $
FRML <> LOSS__HOUSEHOLDS__NORDEA = 0.02*STOCK__HOUSEHOLDS__NORDEA $
FRML <> LOSS__HOUSEHOLDS__IBS = 0.02*STOCK__HOUSEHOLDS__IBS $
FRML <> LOSS__NFC__DANSKE = 0.02*STOCK__NFC__DANSKE $
FRML <> LOSS__NFC__NORDEA = 0.02*STOCK__NFC__NORDEA $
FRML <> LOSS__NFC__IBS = 0.02*STOCK__NFC__IBS $
FRML <> LOSS__RE__DANSKE = 0.02*STOCK__RE__DANSKE $
FRML <> LOSS__RE__NORDEA = 0.02*STOCK__RE__NORDEA $
FRML <> LOSS__RE__IBS = 0.02*STOCK__RE__IBS $
FRML <> LOSS__SOV__DANSKE = 0.02*STOCK__SOV__DANSKE $
FRML <> LOSS__SOV__NORDEA = 0.02*STOCK__SOV__NORDEA $
FRML <> LOSS__SOV__IBS = 0.02*STOCK__SOV__IBS $




### Use of sublist to do conditional Loop

By using 

`DO listname sublist = value $`

the looping will only be performed on the members of list where the value of the corresponding sublist match the value in the do statement.

This can be useful for instance when treating portfolios differently depending on some information. An example could be the different treatment of portfolios depending on regulatory approach. 

In [12]:
mtest='''
    do ports reg = airb  
        REA_W__{ports}= rea({ports}_pd ,{ports}_lgd) 
    enddo  
    do ports reg = sta  
       REA_W__{ports} = {ports}_REA_W(-1) 
    enddo  
 '''
Mexplode(lists+mtest).show


FRML <> REA_W__HOUSEHOLDS = REA(HOUSEHOLDS_PD,HOUSEHOLDS_LGD) $
FRML <> REA_W__NFC = REA(NFC_PD,NFC_LGD) $
FRML <> REA_W__RE = RE_REA_W(-1) $
FRML <> REA_W__SOV = SOV_REA_W(-1) $




In [13]:
mtest='''
doable <> [ports=reg=airb] REA_W__{banks}__{ports} = rea(pd__{banks}__{ports} ,lgd__{banks}__{ports}) 
doable <> [ports=reg=sta] REA_W__{banks}__{ports} = REA_W__{banks}__{ports}(-1) 
 '''
Mexplode(lists+mtest).show




FRML <> REA_W__DANSKE__HOUSEHOLDS = REA(PD__DANSKE__HOUSEHOLDS,LGD__DANSKE__HOUSEHOLDS) $
FRML <> REA_W__DANSKE__NFC = REA(PD__DANSKE__NFC,LGD__DANSKE__NFC) $
FRML <> REA_W__NORDEA__HOUSEHOLDS = REA(PD__NORDEA__HOUSEHOLDS,LGD__NORDEA__HOUSEHOLDS) $
FRML <> REA_W__NORDEA__NFC = REA(PD__NORDEA__NFC,LGD__NORDEA__NFC) $
FRML <> REA_W__IBS__HOUSEHOLDS = REA(PD__IBS__HOUSEHOLDS,LGD__IBS__HOUSEHOLDS) $
FRML <> REA_W__IBS__NFC = REA(PD__IBS__NFC,LGD__IBS__NFC) $
FRML <> REA_W__DANSKE__RE = REA_W__DANSKE__RE(-1) $
FRML <> REA_W__DANSKE__SOV = REA_W__DANSKE__SOV(-1) $
FRML <> REA_W__NORDEA__RE = REA_W__NORDEA__RE(-1) $
FRML <> REA_W__NORDEA__SOV = REA_W__NORDEA__SOV(-1) $
FRML <> REA_W__IBS__RE = REA_W__IBS__RE(-1) $
FRML <> REA_W__IBS__SOV = REA_W__IBS__SOV(-1) $




### Dynamic defined lists to utilize sparsity
Often not all the potential dimensions in a model contains data. For instance all banks don't have positions in all potential countries. 

In order to speed up calculations and to avoid bloating dataframes, we want to avoid calculating and carrying a lot of zeros around. 

This can be achieved by using dynamic naming of lists. For instance create a separate list of countries for each bank. 

The list for each country can be created by using the pandas library to identify in which countries 
there are non-zero positions - straight forward but beyond the scope of this notebook. 

Below is an example where there are three banks which each has exposures in different countries. 

In [18]:
stest='''
DO BANKS 
    do country__{banks}
        value__{banks}__{loancountry} = 42 
    enddo 
    total_{banks} = sum(country__{banks},value__{banks}__{loancountry})  

enddo  
'''
Mexplode(lists +stest).show


FRML <> VALUE__DANSKE__UK = 42 $
FRML <> VALUE__DANSKE__DK = 42 $
FRML <> VALUE__DANSKE__IR = 42 $
FRML <> TOTAL_DANSKE = (VALUE__DANSKE__UK+VALUE__DANSKE__DK+VALUE__DANSKE__IR) $
FRML <> VALUE__NORDEA__SE = 42 $
FRML <> VALUE__NORDEA__DK = 42 $
FRML <> TOTAL_NORDEA = (VALUE__NORDEA__SE+VALUE__NORDEA__DK) $
FRML <> VALUE__IBS__DK = 42 $
FRML <> VALUE__IBS__IR = 42 $
FRML <> TOTAL_IBS = (VALUE__IBS__DK+VALUE__IBS__IR) $




In [ ]:
list_extract(mtest)

## Sum, sum over list 
sum(LIST,expression) = sums expression over the elements of a list substitution the sublist names.  

In [20]:
mtest = ''' 
loss_total =  sum(banks,sum(ports,loss__{banks}__{ports}))
'''

Mexplode(lists+ mtest).show

FRML <> LOSS_TOTAL = ((LOSS__DANSKE__HOUSEHOLDS+LOSS__DANSKE__NFC+LOSS__DANSKE__RE+LOSS__DANSKE__SOV)+(LOSS__NORDEA__HOUSEHOLDS+LOSS__NORDEA__NFC+LOSS__NORDEA__RE+LOSS__NORDEA__SOV)+(LOSS__IBS__HOUSEHOLDS+LOSS__IBS__NFC+LOSS__IBS__RE+LOSS__IBS__SOV)) $




## Sum, conditional sums 
sum(LIST,expression) = sums expression over the elements of a list substitution the sublist names.  

In [23]:
mtest = ''' 
loss_total__sweden  =  sum(banks country=sweden,sum(ports,loss__{banks}__{country}__{ports}))
loss_total__denmark =  sum(banks country=denmark,sum(ports,loss__{banks}__{country}__{ports}))
'''

Mexplode(lists+mtest).show

FRML <> LOSS_TOTAL__SWEDEN = ((LOSS__NORDEA__SWEDEN__HOUSEHOLDS+LOSS__NORDEA__SWEDEN__NFC+LOSS__NORDEA__SWEDEN__RE+LOSS__NORDEA__SWEDEN__SOV)) $
FRML <> LOSS_TOTAL__DENMARK = ((LOSS__DANSKE__DENMARK__HOUSEHOLDS+LOSS__DANSKE__DENMARK__NFC+LOSS__DANSKE__DENMARK__RE+LOSS__DANSKE__DENMARK__SOV)+(LOSS__IBS__DENMARK__HOUSEHOLDS+LOSS__IBS__DENMARK__NFC+LOSS__IBS__DENMARK__RE+LOSS__IBS__DENMARK__SOV)) $




## Doable: A little more high level 

All the do .. enddo statements introduces some visual noise. 
To handle the situation, where we just want to loop over some indexes and do calculations the **doable** function can handle some of the details. It will extract the indicies from the left hand side and loop over these indicies. 
ALso if the option sum= is provided the the sums of all the resulting left hand sides will be calculated. 


In [ ]:
frml = '''
list sectors = sectors : a b 
list banks   = banks : hest ko 

doable <> loss__{banks}__{sectors} =holding__{banks}__{sectors} * pd__{banks}__{sectors}
doable <sum= all>    b__{sectors}__{banks}  = b
doable <sum= all>    diff(xx__{sectors}__{banks})  = 42

        '''.upper()
xx = Mexplode(frml)

xx.show
